In [2]:
##########################################################################################
# Import library
import sys
import os
import json
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()

from plotly import tools
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go

from sklearn import model_selection, preprocessing, metrics
import lightgbm as lgb
sys.path.append('.')
sys.path.append('./code/feature')
from kit import *
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from timeit import time
pd.set_option('display.max_columns', 500)

/home/ubuntu/anaconda3/lib/python3.6/site-packages/matplotlib/font_manager.py:278: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  'Matplotlib is building the font cache using fc-list. '


In [39]:
# load pkl file 
train_df = pd.read_pickle('./data/train.pkl')
test_df = pd.read_pickle('./data/test.pkl')

In [40]:
# we select only 100k records for mac version training
max_num = 50000
train_df = train_df[:max_num]
test_df = test_df[:max_num]
print(train_df.shape)
print(test_df.shape)
print(train_df.columns)

(50000, 39)
(50000, 39)
Index(['channelGrouping', 'date', 'fullVisitorId', 'visitId', 'visitNumber',
       'visitStartTime', 'device_browser', 'device_deviceCategory',
       'device_isMobile', 'device_operatingSystem', 'geoNetwork_city',
       'geoNetwork_continent', 'geoNetwork_country', 'geoNetwork_metro',
       'geoNetwork_networkDomain', 'geoNetwork_region',
       'geoNetwork_subContinent', 'totals_bounces', 'totals_hits',
       'totals_newVisits', 'totals_pageviews', 'totals_sessionQualityDim',
       'totals_timeOnSite', 'totals_totalTransactionRevenue',
       'totals_transactionRevenue', 'totals_transactions',
       'trafficSource_adContent',
       'trafficSource_adwordsClickInfo.adNetworkType',
       'trafficSource_adwordsClickInfo.gclId',
       'trafficSource_adwordsClickInfo.isVideoAd',
       'trafficSource_adwordsClickInfo.page',
       'trafficSource_adwordsClickInfo.slot', 'trafficSource_campaign',
       'trafficSource_isTrueDirect', 'trafficSource_keyword',
 

In [41]:
# category different features 
train_df.to_csv("./data/train.csv",index=False)
test_df.to_csv("./data/test.csv",index=False)


In [17]:
train_df.head(10).T

,0,1,2,3,4,5,6,7,8,9
channelGrouping,Organic Search,Referral,Direct,Organic Search,Organic Search,Referral,Referral,Organic Search,Organic Search,Organic Search
date,20171016,20171016,20171016,20171016,20171016,20171016,20171016,20171016,20171016,20171016
fullVisitorId,3162355547410993243,8934116514970143966,7992466427990357681,9075655783635761930,6960673291025684308,0166277907528479249,8349655975937271469,1332629902468998662,632878546807742341,1259490915281096752
visitId,1508198450,1508176307,1508201613,1508169851,1508190552,1508196701,1508152478,1508206208,1508207516,1508165159
visitNumber,1,6,1,1,1,1,1,1,1,2
visitStartTime,1508198450,1508176307,1508201613,1508169851,1508190552,1508196701,1508152478,1508206208,1508207516,1508165159
device_browser,Firefox,Chrome,Chrome,Chrome,Chrome,Chrome,Chrome,Chrome,Chrome,Safari
device_deviceCategory,desktop,desktop,mobile,desktop,desktop,desktop,desktop,desktop,desktop,mobile
device_isMobile,False,False,True,False,False,False,False,False,False,True
device_operatingSystem,Windows,Chrome OS,Android,Windows,Windows,Macintosh,Macintosh,Windows,Macintosh,iOS


In [37]:
feat = 'customDimensions_value'
print(train_df[feat].unique())
print(train_df[feat].unique().shape)

['EMEA' 'North America' 'Central America' nan 'APAC' 'South America']
(6,)
